In [0]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
%matplotlib inline

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#list(drive.ListFile())

In [0]:
file_id = '1wj5EXCBZ9wyzxYoyHk3XHuvV2ZZSTGAb'
downloaded = drive.CreateFile({'id': file_id})

In [0]:
downloaded.GetContentFile('fer2013.csv')  

In [7]:
dff = df = pd.DataFrame.from_csv('fer2013.csv', index_col=None)
df.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


(35887, 3)

In [0]:
train = df[0:28709]
cv = df[28709:32298] #PublicTest 28709,32297
test = df[32298:] #PrivateTest

In [0]:
y_train = train.emotion
y_cv = cv.emotion
y_test = test.emotion

In [0]:
train_size = 28709
cv_size = test_size = 3589
num_pix = 2304 #48*48
nparr = np.zeros((train_size, num_pix))
nparr1 = np.zeros((cv_size, num_pix))
nparr2 = np.zeros((test_size, num_pix))

In [0]:
for i in range(train_size):
    b = [int(j) for j in train.pixels[i].split()]
    nparr[i] = np.array(b)

In [0]:
maindf = pd.DataFrame(nparr, columns=[int(i) for i in range(num_pix)])
X = maindf

In [0]:
for i in range(28709,32298):#public
  b = [int(j) for j in cv.pixels[i].split()]
  nparr1[i-28709] = np.array(b)

In [0]:
maincv = pd.DataFrame(nparr1, columns=[int(i) for i in range(2304)])
X_cv = maincv

In [0]:
#for i in range(32298,35887):#private
 #   test.pixels[i] = [int(j) for j in test.pixels[i].split()]
#   nparr2[i-32298] = np.array(test.pixels[i])

In [0]:
#maintest = pd.DataFrame(nparr2, columns=[int(i) for i in range(num_pix)])
#X_test = maintest

In [0]:
import tensorflow as tf

In [0]:
a = dff.pixels
arr = np.zeros((train_size, num_pix))
for i in range (train_size):
    b = [int(j) for j in a[i].split()]
    arr[i] = np.array(b)


In [19]:
x = tf.placeholder('float' , [None , num_pix])
y = tf.placeholder('float')

from keras.utils.np_utils import to_categorical

y_train = to_categorical(y_train, 7)

Using TensorFlow backend.


In [0]:
n_h1 = train_size
n_h2 = 2000
n_h3 = 2000
n_h4 = 2000
n_h5 = 2000
n_h6 = 2000
n_classes = 7
batch_size = 3072 #1024 #2048
def neural_network(data):
    hidden_l_1 = {'weights' :tf.Variable(tf.random_normal([num_pix , n_h1])) , 'bias':tf.Variable(tf.random_normal([n_h1]))}
    hidden_l_2 = {'weights' :tf.Variable(tf.random_normal([n_h1 , n_h2])) , 'bias':tf.Variable(tf.random_normal([n_h2]))}
    hidden_l_3 = {'weights' :tf.Variable(tf.random_normal([n_h2 , n_h3])) , 'bias':tf.Variable(tf.random_normal([n_h3]))}
    hidden_l_4 = {'weights' :tf.Variable(tf.random_normal([n_h3 , n_h4])) , 'bias':tf.Variable(tf.random_normal([n_h4]))}
    hidden_l_5 = {'weights' :tf.Variable(tf.random_normal([n_h4 , n_h5])) , 'bias':tf.Variable(tf.random_normal([n_h5]))}
    hidden_l_6 = {'weights' :tf.Variable(tf.random_normal([n_h5 , n_h6])) , 'bias':tf.Variable(tf.random_normal([n_h6]))}
    output_l = {'weights' :tf.Variable(tf.random_normal([n_h6 , n_classes])) , 'bias':tf.Variable(tf.random_normal([n_classes]))}
    l1 = tf.add(tf.matmul(data,hidden_l_1['weights']) , hidden_l_1['bias'])    
    l1 = tf.nn.relu(l1)
    l2 = tf.add(tf.matmul(l1,hidden_l_2['weights']) , hidden_l_2['bias'])    
    l2 = tf.nn.relu(l2)
    l3 = tf.add(tf.matmul(l2,hidden_l_3['weights']) , hidden_l_3['bias'])    
    l3 = tf.nn.relu(l3)
    l4 = tf.add(tf.matmul(l3,hidden_l_4['weights']) , hidden_l_4['bias'])    
    l4 = tf.nn.relu(l4)
    l5 = tf.add(tf.matmul(l4,hidden_l_5['weights']) , hidden_l_5['bias'])    
    l5 = tf.nn.relu(l5)
    l6 = tf.add(tf.matmul(l5,hidden_l_6['weights']) , hidden_l_6['bias'])    
    l6 = tf.nn.relu(l6)
    output = tf.matmul(l6,output_l['weights']) + output_l['bias']   
    return output

In [0]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [0]:
def training(X, epc):
    prediction = neural_network(X)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction , labels=y))
    #                       learning_rate = 0.001
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    #num_epochs = epc
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epochs in range(epc):
            epochs_loss = 0
            for i in range(train_size // batch_size):
                #ex, ey = next_batch(batch_size, arr, np.array(y_train))
                ex = arr[i*batch_size:(i+1)*batch_size]
                ey = y_train[i*batch_size:(i+1)*batch_size]
                _, c = sess.run([optimizer, cost], feed_dict={
                    X: ex, y: ey})    
                epochs_loss += c
            correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
            print('Epoch', epochs, '/', epc, 'loss', epochs_loss, '   ', 'Accuracy:', accuracy.eval({X: X_cv,
                                  y: to_categorical(y_cv, 7)},sess), '  ','Accuracy:', accuracy.eval({X: arr,y: y_train},sess))
        
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:', accuracy.eval({X: arr,
                                          y: y_train}))
        saver = tf.train.Saver()
        save_path = saver.save(sess, "./model.ckpt")

In [23]:
  training(x, 50) #3 hidden layers: trainsize, 3000, 3000/ 50 epochs/  batchsize 2048

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Epoch 0 / 50 loss 43590111104.0     Accuracy: 0.18361661    Accuracy: 0.1724198
Epoch 1 / 50 loss 15624507136.0     Accuracy: 0.26023963    Accuracy: 0.26535234
Epoch 2 / 50 loss 6057901824.0     Accuracy: 0.29395375    Accuracy: 0.28966525
Epoch 3 / 50 loss 3155013600.0     Accuracy: 0.29116747    Accuracy: 0.30882302
Epoch 4 / 50 loss 2070319616.0     Accuracy: 0.2897743    Accuracy: 0.3132119
Epoch 5 / 50 loss 1649916968.0     Accuracy: 0.31178603    Accuracy: 0.34327215
Epoch 6 / 50 loss 1450280296.0     Accuracy: 0.2917247    Accuracy: 0.33996308
Epoch 7 / 50 loss 1318479152.0     Accuracy: 0.30147675    Accuracy: 0.34978577
Epoch 8 / 50 loss 1405086424.0     Accuracy: 0.20869324    Accuracy: 0.2800864
Epoch 9 / 50 loss 2550614136.0     Accuracy: 0.20228475    Accuracy: 0.23090321
Epoch 10 / 50 

Epoch 45 / 50 loss 260285333.0     Accuracy: 0.34745055    Accuracy: 0.7162214
Epoch 46 / 50 loss 210328321.0     Accuracy: 0.34745055    Accuracy: 0.67891604
Epoch 47 / 50 loss 281329248.0     Accuracy: 0.3719699    Accuracy: 0.6876589
Epoch 48 / 50 loss 423555360.0     Accuracy: 0.2992477    Accuracy: 0.57630014
Epoch 49 / 50 loss 329242731.0     Accuracy: 0.33129007    Accuracy: 0.6749451
Accuracy: 0.6749451


In [25]:
training(x,100) #6 hidden layers: trainsize, 3000, 3000, 2000, 1000, 1000/ 100 epochs  batchsize 2048

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Epoch 0 / 100 loss 367254041526272.0     Accuracy: 0.2504876    Accuracy: 0.25288236
Epoch 1 / 100 loss 112965325160448.0     Accuracy: 0.25410977    Accuracy: 0.25761956
Epoch 2 / 100 loss 46366568873984.0     Accuracy: 0.21454445    Accuracy: 0.23351562
Epoch 3 / 100 loss 25498658799616.0     Accuracy: 0.27027026    Accuracy: 0.2960744
Epoch 4 / 100 loss 17731365568512.0     Accuracy: 0.235163    Accuracy: 0.26277474
Epoch 5 / 100 loss 13944069095424.0     Accuracy: 0.2733352    Accuracy: 0.32094464
Epoch 6 / 100 loss 12635646263296.0     Accuracy: 0.28169405    Accuracy: 0.3361315
Epoch 7 / 100 loss 11517198401536.0     Accuracy: 0.29618278    Accuracy: 0.3473127
Epoch 8 / 100 loss 12492860751872.0     Accuracy: 0.29283923    Accuracy: 0.35967815
Epoch 9 / 100 loss 14362438402048.0     Accuracy: 0

Epoch 43 / 100 loss 3216827695104.0     Accuracy: 0.31122875    Accuracy: 0.6219304
Epoch 44 / 100 loss 2627347234816.0     Accuracy: 0.33630538    Accuracy: 0.7471873
Epoch 45 / 100 loss 2325275705344.0     Accuracy: 0.32933965    Accuracy: 0.62067646
Epoch 46 / 100 loss 2514539896832.0     Accuracy: 0.3070493    Accuracy: 0.5971995
Epoch 47 / 100 loss 2430414348288.0     Accuracy: 0.35998887    Accuracy: 0.7753666
Epoch 48 / 100 loss 1626106740736.0     Accuracy: 0.33240458    Accuracy: 0.7634888
Epoch 49 / 100 loss 1189608480768.0     Accuracy: 0.35831708    Accuracy: 0.7384792
Epoch 50 / 100 loss 1131645267968.0     Accuracy: 0.34633604    Accuracy: 0.7357623
Epoch 51 / 100 loss 1293656731648.0     Accuracy: 0.33881304    Accuracy: 0.7422411
Epoch 52 / 100 loss 1428747616256.0     Accuracy: 0.32850376    Accuracy: 0.78306454
Epoch 53 / 100 loss 1724355080192.0     Accuracy: 0.35887435    Accuracy: 0.78783655
Epoch 54 / 100 loss 1492704202752.0     Accuracy: 0.3527445    Accuracy: 0

Epoch 91 / 100 loss 735782895616.0     Accuracy: 0.361382    Accuracy: 0.85137063
Epoch 92 / 100 loss 857160456192.0     Accuracy: 0.37141266    Accuracy: 0.87164307
Epoch 93 / 100 loss 966020259840.0     Accuracy: 0.3661187    Accuracy: 0.8366714
Epoch 94 / 100 loss 1008122765312.0     Accuracy: 0.33324045    Accuracy: 0.72757673
Epoch 95 / 100 loss 1090570088448.0     Accuracy: 0.38116467    Accuracy: 0.8599394
Epoch 96 / 100 loss 713622702080.0     Accuracy: 0.3318473    Accuracy: 0.8150754
Epoch 97 / 100 loss 670109005824.0     Accuracy: 0.319309    Accuracy: 0.7988784
Epoch 98 / 100 loss 718429051904.0     Accuracy: 0.31819448    Accuracy: 0.7752621
Epoch 99 / 100 loss 886377760768.0     Accuracy: 0.34076345    Accuracy: 0.8076561
Accuracy: 0.8076561


In [22]:
training(x, 150) #6 hidden layers:trainszie & others 2000/ 150 epochs/ batchsize 2048

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Epoch 0 / 150 loss 568287094112256.0     Accuracy: 0.19002508    Accuracy: 0.17956042
Epoch 1 / 150 loss 127385026953216.0     Accuracy: 0.2610755    Accuracy: 0.26399386
Epoch 2 / 150 loss 72394938253312.0     Accuracy: 0.21955976    Accuracy: 0.22355358
Epoch 3 / 150 loss 39223367434240.0     Accuracy: 0.24547228    Accuracy: 0.26138145
Epoch 4 / 150 loss 27327851528192.0     Accuracy: 0.25606018    Accuracy: 0.29746768
Epoch 5 / 150 loss 22784400031744.0     Accuracy: 0.25438842    Accuracy: 0.30767354
Epoch 6 / 150 loss 24098288304128.0     Accuracy: 0.27946502    Accuracy: 0.3338326
Epoch 7 / 150 loss 23850762108928.0     Accuracy: 0.30064085    Accuracy: 0.3739594
Epoch 8 / 150 loss 22385642307584.0     Accuracy: 0.23544163    Accuracy: 0.3001846
Epoch 9 / 150 loss 20468617707520.0     Accuracy

Epoch 43 / 150 loss 5182749736960.0     Accuracy: 0.33324045    Accuracy: 0.58441603
Epoch 44 / 150 loss 4679286112256.0     Accuracy: 0.34828642    Accuracy: 0.6596538
Epoch 45 / 150 loss 2719001174016.0     Accuracy: 0.35692394    Accuracy: 0.7431119
Epoch 46 / 150 loss 3401254912000.0     Accuracy: 0.34522152    Accuracy: 0.70908076
Epoch 47 / 150 loss 5999988064256.0     Accuracy: 0.3059348    Accuracy: 0.6052109
Epoch 48 / 150 loss 5309776068608.0     Accuracy: 0.29478964    Accuracy: 0.62234837
Epoch 49 / 150 loss 3768483545088.0     Accuracy: 0.30482027    Accuracy: 0.62510014
Epoch 50 / 150 loss 3406648541184.0     Accuracy: 0.35246587    Accuracy: 0.7992964
Epoch 51 / 150 loss 1955127672832.0     Accuracy: 0.36723322    Accuracy: 0.8116967
Epoch 52 / 150 loss 1347203203072.0     Accuracy: 0.3480078    Accuracy: 0.8244453
Epoch 53 / 150 loss 1263470264320.0     Accuracy: 0.35608804    Accuracy: 0.82005644
Epoch 54 / 150 loss 1223939694592.0     Accuracy: 0.35330176    Accuracy:

Epoch 91 / 150 loss 2766919204864.0     Accuracy: 0.34104207    Accuracy: 0.73496115
Epoch 92 / 150 loss 2268333850624.0     Accuracy: 0.36166063    Accuracy: 0.76408094
Epoch 93 / 150 loss 2812251545600.0     Accuracy: 0.361382    Accuracy: 0.77400815
Epoch 94 / 150 loss 2765296246784.0     Accuracy: 0.37029815    Accuracy: 0.7868961
Epoch 95 / 150 loss 2661015207936.0     Accuracy: 0.36974087    Accuracy: 0.81949914
Epoch 96 / 150 loss 1971115253760.0     Accuracy: 0.38478684    Accuracy: 0.8489672
Epoch 97 / 150 loss 1567045435392.0     Accuracy: 0.36890498    Accuracy: 0.8657912
Epoch 98 / 150 loss 847119306752.0     Accuracy: 0.36890498    Accuracy: 0.8960953
Epoch 99 / 150 loss 516860203008.0     Accuracy: 0.38005015    Accuracy: 0.91891044
Epoch 100 / 150 loss 459657730048.0     Accuracy: 0.37308443    Accuracy: 0.9102024
Epoch 101 / 150 loss 624675147776.0     Accuracy: 0.37224853    Accuracy: 0.9117002
Epoch 102 / 150 loss 580964063232.0     Accuracy: 0.36221787    Accuracy: 0

Epoch 139 / 150 loss 557726297088.0     Accuracy: 0.3488437    Accuracy: 0.8657215
Epoch 140 / 150 loss 461751539712.0     Accuracy: 0.3747562    Accuracy: 0.92841965
Epoch 141 / 150 loss 577191495680.0     Accuracy: 0.37308443    Accuracy: 0.9064753
Epoch 142 / 150 loss 432522502144.0     Accuracy: 0.36779046    Accuracy: 0.9238566
Epoch 143 / 150 loss 631774695424.0     Accuracy: 0.36834773    Accuracy: 0.90772927
Epoch 144 / 150 loss 536411943936.0     Accuracy: 0.37726387    Accuracy: 0.935003
Epoch 145 / 150 loss 650707536896.0     Accuracy: 0.36221787    Accuracy: 0.8964436
Epoch 146 / 150 loss 542372388864.0     Accuracy: 0.36249653    Accuracy: 0.9120833
Epoch 147 / 150 loss 754130415616.0     Accuracy: 0.38311508    Accuracy: 0.90940124
Epoch 148 / 150 loss 596806535168.0     Accuracy: 0.378657    Accuracy: 0.9237521
Epoch 149 / 150 loss 740047366144.0     Accuracy: 0.36277515    Accuracy: 0.8795848
Accuracy: 0.8795848


In [24]:
training(x, 100) #6 hidden layers:trainszie & others 2000/ 100 epochs/ batchsize 1024

Epoch 0 / 100 loss 636695020044288.0     Accuracy: 0.1816662    Accuracy: 0.18823366
Epoch 1 / 100 loss 133536592953344.0     Accuracy: 0.26330453    Accuracy: 0.28109652
Epoch 2 / 100 loss 61133272121344.0     Accuracy: 0.30091947    Accuracy: 0.33181232
Epoch 3 / 100 loss 49110426320896.0     Accuracy: 0.2752856    Accuracy: 0.32780662
Epoch 4 / 100 loss 46546572017664.0     Accuracy: 0.2513235    Accuracy: 0.3367585
Epoch 5 / 100 loss 30647151165440.0     Accuracy: 0.29506826    Accuracy: 0.40457696
Epoch 6 / 100 loss 24526555119616.0     Accuracy: 0.28698802    Accuracy: 0.4210178
Epoch 7 / 100 loss 45881168297984.0     Accuracy: 0.2783505    Accuracy: 0.38183147
Epoch 8 / 100 loss 34811285733376.0     Accuracy: 0.30565616    Accuracy: 0.40109372
Epoch 9 / 100 loss 26945691123712.0     Accuracy: 0.29869044    Accuracy: 0.44780383
Epoch 10 / 100 loss 26718660001792.0     Accuracy: 0.25494567    Accuracy: 0.3988993
Epoch 11 / 100 loss 23151983853568.0     Accuracy: 0.26553357    Accu

Epoch 49 / 100 loss 6757832523776.0     Accuracy: 0.35803846    Accuracy: 0.7387579
Epoch 50 / 100 loss 5660119793664.0     Accuracy: 0.3585957    Accuracy: 0.73193073
Epoch 51 / 100 loss 4548372709376.0     Accuracy: 0.36946225    Accuracy: 0.774461
Epoch 52 / 100 loss 3465382445056.0     Accuracy: 0.36723322    Accuracy: 0.8292173
Epoch 53 / 100 loss 2448008302592.0     Accuracy: 0.3716913    Accuracy: 0.84370756
Epoch 54 / 100 loss 1696877350912.0     Accuracy: 0.36974087    Accuracy: 0.8461806
Epoch 55 / 100 loss 1473822920704.0     Accuracy: 0.37614936    Accuracy: 0.85882473
Epoch 56 / 100 loss 1518358069248.0     Accuracy: 0.37447757    Accuracy: 0.8708767
Epoch 57 / 100 loss 1640622211072.0     Accuracy: 0.37782112    Accuracy: 0.8562472
Epoch 58 / 100 loss 1460276666368.0     Accuracy: 0.36639732    Accuracy: 0.8635619
Epoch 59 / 100 loss 1707772116992.0     Accuracy: 0.37224853    Accuracy: 0.84586716
Epoch 60 / 100 loss 1868472967168.0     Accuracy: 0.36751184    Accuracy: 0

Epoch 98 / 100 loss 1893685362688.0     Accuracy: 0.3496796    Accuracy: 0.8176182
Epoch 99 / 100 loss 1763097159680.0     Accuracy: 0.36834773    Accuracy: 0.84210527
Accuracy: 0.84210527


In [22]:
training(x, 100) #6 hidden layers:trainszie & others 2000/ 100 epochs/ batchsize 3072

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Epoch 0 / 100 loss 492581072928768.0     Accuracy: 0.13151295    Accuracy: 0.14291686
Epoch 1 / 100 loss 185564222980096.0     Accuracy: 0.17804402    Accuracy: 0.17252429
Epoch 2 / 100 loss 85334469640192.0     Accuracy: 0.19086096    Accuracy: 0.20523182
Epoch 3 / 100 loss 56979365298176.0     Accuracy: 0.19866258    Accuracy: 0.20331603
Epoch 4 / 100 loss 41111645323264.0     Accuracy: 0.1950404    Accuracy: 0.21338256
Epoch 5 / 100 loss 27762928386048.0     Accuracy: 0.25020897    Accuracy: 0.2767425
Epoch 6 / 100 loss 20979080626176.0     Accuracy: 0.2646977    Accuracy: 0.29311365
Epoch 7 / 100 loss 16505719881728.0     Accuracy: 0.27946502    Accuracy: 0.3079174
Epoch 8 / 100 loss 14872712052736.0     Accuracy: 0.29757592    Accuracy: 0.33602703
Epoch 9 / 100 loss 13076313014272.0     Accuracy

Epoch 43 / 100 loss 3571299516416.0     Accuracy: 0.29980496    Accuracy: 0.56163573
Epoch 44 / 100 loss 3547704918016.0     Accuracy: 0.31401506    Accuracy: 0.65885264
Epoch 45 / 100 loss 2526218911744.0     Accuracy: 0.31624407    Accuracy: 0.66048974
Epoch 46 / 100 loss 2532957552640.0     Accuracy: 0.32655334    Accuracy: 0.6553346
Epoch 47 / 100 loss 2879960465408.0     Accuracy: 0.326832    Accuracy: 0.64460623
Epoch 48 / 100 loss 2622302076928.0     Accuracy: 0.33407634    Accuracy: 0.68724096
Epoch 49 / 100 loss 2062865022976.0     Accuracy: 0.33351907    Accuracy: 0.68522066
Epoch 50 / 100 loss 1902304198656.0     Accuracy: 0.3346336    Accuracy: 0.679543
Epoch 51 / 100 loss 2096464052224.0     Accuracy: 0.3471719    Accuracy: 0.7161517
Epoch 52 / 100 loss 2583549263872.0     Accuracy: 0.32348844    Accuracy: 0.60597724
Epoch 53 / 100 loss 3093946269696.0     Accuracy: 0.33686262    Accuracy: 0.7119719
Epoch 54 / 100 loss 2133115043840.0     Accuracy: 0.34187797    Accuracy: 

Epoch 92 / 100 loss 453473054720.0     Accuracy: 0.33351907    Accuracy: 0.85231113
Epoch 93 / 100 loss 504677523456.0     Accuracy: 0.35413763    Accuracy: 0.8979414
Epoch 95 / 100 loss 358487027712.0     Accuracy: 0.35748118    Accuracy: 0.9047685
Epoch 96 / 100 loss 318653429760.0     Accuracy: 0.36556143    Accuracy: 0.84384686
Epoch 97 / 100 loss 382161924096.0     Accuracy: 0.3700195    Accuracy: 0.8719565
Epoch 98 / 100 loss 421301616640.0     Accuracy: 0.34382835    Accuracy: 0.89936954
Epoch 99 / 100 loss 418318217216.0     Accuracy: 0.31317917    Accuracy: 0.80100316
Accuracy: 0.80100316
